# Agents
By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. 
Agents are systems that use an LLM as a reasoning enginer to determine which actions to take and what the inputs to those actions should be. 
The results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.

In [15]:
# Import relevant functionality
import os
import openai
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
openai.api_base = os.getenv('OPENAI_API_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")



In [20]:
# Create the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name="gpt4o",  # replace with your deployment name
    temperature=0.5,
    max_tokens=100,
    openai_api_version="2023-03-15-preview"
)

# Create the search tool
search = TavilySearchResults(max_results=2)
tools = [search]

# Create the agent with memory
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Configuration for conversation thread
config = {"configurable": {"thread_id": "abc123"}}

# Use the agent for conversation
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content="Hi Bob! It's great to meet you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 88, 'total_tokens': 105}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5f4bad809a', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-9c71352a-61c5-412e-99c9-0d583165e271-0', usage_metadata={'input_tokens': 88, 'output_tokens': 17, 'total_tokens': 105})]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_a5q1HIcyOd67ridtGlZk5TY6', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 120, 'total_tokens': 142}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5f4bad809a', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results':